In [1]:
import pandas as pd
import numpy as np
from typing import Any
try:
    from typing import Self
except ImportError:
    from typing_extensions import Self

In [2]:
def array_to_str(arr):
    if arr.ndim == 1:
        return ','.join(map(str, np.around(arr, 4).tolist()))
    elif arr.ndim == 2:
        return '\n'.join((','.join(map(str, x)) for x in np.around(arr, 4).tolist()))
    else:
        raise ValueError('Only 1 and 2-d arrays suppported')

def write(file: str, data: Any):
    with open(file, 'w') as f:
        f.write(data if isinstance(data, str) else str(data))

def rescale(x: np.ndarray) -> np.ndarray:
    min_ = np.min(x)
    range_ = np.ptp(x)
    return (x - min_) / range_

In [3]:
df = pd.read_csv('time_series_covid19_deaths_US_raw.csv')
raw_data = df.to_numpy()

In [4]:
cum_data = np.cumsum(raw_data, axis=0)
data = np.diff(cum_data)

del raw_data

In [5]:
# wisc_cal_data = cum_data[(4, 48), :]
# q1_str = array_to_str(wisc_cal_data)
# write('q1.txt', q1_str)

# del cum_data
# del q1_str

In [6]:
# q2_data = data[(4, 48), :] # 4-> cal, 48 -> wisco
# q2_str = array_to_str(q2_data)
# write('q2.txt', q2_str)

# del q2_data
# del q2_str

In [7]:
# write('q3.txt', '''Mean of the time-differenced data
# Standard deviation of the time-differenced data
# Median of the time-differenced data
# Linear trend coefficient of the data
# Auto-correlation of the data''')

In [8]:
mean = np.mean(data, axis=1)
mean = rescale(mean)

In [9]:
std = np.std(data, axis=1)
std = rescale(std)

In [10]:
median = np.median(data, axis=1)
median = rescale(median)

In [11]:
ltc = np.array([sum((x[i] - mean[idx]) * (i + 1 - (half_length := ((data.shape[-1] + 1) / 2))) for i in range(len(x))) for idx, x in enumerate(data.tolist())]) / sum((i+1 - half_length) ** 2 for i in range(data.shape[-1]))
ltc = rescale(ltc)

In [12]:
ac = np.array([ sum((x[i] * x[i-1] for i in range(1, len(x)))) / np.sum(np.power(x, 2)) for x in (data - mean.reshape(50, -1)).tolist()])
ac = rescale(ac)

In [13]:
parametric_data = np.empty((50, 5))
parametric_data[:, 0] = mean
parametric_data[:, 1] = std
parametric_data[:, 2] = median
parametric_data[:, 3] = ltc
parametric_data[:, 4] = ac

# q4_str = array_to_str(parametric_data)
# write('q4.txt', q4_str)
# del q4_str

In [14]:
class HierarchicalClustering:
    class Clustering:
        @staticmethod
        def _compute_distances(X: np.ndarray) -> np.ndarray:
            n = X.shape[0]
            distance_matrix = np.zeros((n, n))
        
            for i in range(n):
                for j in range(i + 1, n):
                    distance_matrix[i, j] = np.linalg.norm(X[i] - X[j])
        
            distance_matrix += distance_matrix.T
        
            return distance_matrix
        
        @staticmethod
        def _single(distance_matrix: np.ndarray, num_clusters: int) -> list:
            num_points = distance_matrix.shape[0]
            clusters = [[i] for i in range(num_points)]
        
            while len(clusters) > num_clusters:
                # Find the two closest clusters
                min_distance = np.inf
                merge_index1, merge_index2 = None, None
        
                for i in range(len(clusters)):
                    for j in range(i + 1, len(clusters)):
                        cluster1 = clusters[i]
                        cluster2 = clusters[j]
        
                        # Find the minimum distance between the two clusters
                        distance = np.min(distance_matrix[np.ix_(cluster1, cluster2)])
        
                        if distance < min_distance:
                            min_distance = distance
                            merge_index1, merge_index2 = i, j
        
                # Merge the two closest clusters
                clusters[merge_index1].extend(clusters[merge_index2])
                del clusters[merge_index2]
        
                # Update the distance matrix with the new distances from the merged cluster
                combined_cluster = clusters[merge_index1]
                for i in range(distance_matrix.shape[0]):
                    if i in combined_cluster:
                        continue
                    distance = np.min(distance_matrix[np.ix_(combined_cluster, [i])])
                    distance_matrix[i, combined_cluster] = distance
                    distance_matrix[combined_cluster, i] = distance
        
            return tuple(map(tuple, clusters))
    
        @staticmethod
        def _complete(distance_matrix: np.ndarray, n_clusters: int) -> list:
            num_points = distance_matrix.shape[0]
            clusters = [[i] for i in range(num_points)]
        
            while len(clusters) > n_clusters:
                # Find the two clusters with the smallest maximum distance
                min_max_distance = np.inf
                merge_index1, merge_index2 = None, None
        
                for i in range(len(clusters)):
                    for j in range(i + 1, len(clusters)):
                        cluster1 = clusters[i]
                        cluster2 = clusters[j]
        
                        # Find the maximum distance between the two clusters
                        max_distance = np.max(distance_matrix[np.ix_(cluster1, cluster2)])
        
                        if max_distance < min_max_distance:
                            min_max_distance = max_distance
                            merge_index1, merge_index2 = i, j
        
                # Merge the two closest clusters
                clusters[merge_index1].extend(clusters[merge_index2])
                del clusters[merge_index2]
        
                # Update the distance matrix with the new distances from the merged cluster
                combined_cluster = clusters[merge_index1]
                for i in range(distance_matrix.shape[0]):
                    if i in combined_cluster:
                        continue
                    distance = np.max(distance_matrix[np.ix_(combined_cluster, [i])])
                    distance_matrix[i, combined_cluster] = distance
                    distance_matrix[combined_cluster, i] = distance
        
            return clusters
        
    _METHODS = {
        'single': Clustering._single,
        'complete': Clustering._complete,
    }

    def __init__(self, n_clusters: int = 2, linkage: str = 'single'):
        self._n_clusters = n_clusters
        
        assert HierarchicalClustering._is_linkage_valid(linkage), (
            f'Invalid Linkage, valid linkages are \'single\' and '
            f'\'complete\', found {linkage!r}'
        )
        
        self._linkage = linkage

    def _build(self, X: np.ndarray):
        distance_matrix = HierarchicalClustering.Clustering._compute_distances(X)
        cluster_func = HierarchicalClustering._METHODS.get(self._linkage)
        if cluster_func is None:
            raise ValueError(
                f'Invalid Linkage, the Hierarchical Clustering instance '
                f'may have been tampered with. found {self.linkage!r}'
            )

        self._clusters = cluster_func(distance_matrix, self._n_clusters)

    def fit(self, X: np.ndarray) -> Self:
        self._build(X)
        return self

    def predict(self, X: np.ndarray):
        pass
    
    @property
    def clusters(self):
        return self._clusters

    @clusters.setter
    def clusters(self, value):
        raise ValueError('The `clusters` attribute should not be modified externally!')
    
    @staticmethod
    def _is_linkage_valid(linkage: str) -> bool:
        return linkage in HierarchicalClustering._METHODS.keys()

In [15]:
slhc = HierarchicalClustering(n_clusters = 5)
slhc = slhc.fit(parametric_data)

idxs = np.arange(parametric_data.shape[0])

for cluster_idx, cluster in enumerate(slhc.clusters):
    for item in cluster:
        idxs[item] = cluster_idx

# q5_str = array_to_str(idxs)
# write('q5.txt', q5_str)
# del q5_str

In [16]:
clhc = HierarchicalClustering(n_clusters = 5, linkage='complete')
clhc = clhc.fit(parametric_data)

idxs = np.arange(parametric_data.shape[0])

for cluster_idx, cluster in enumerate(clhc.clusters):
    for item in cluster:
        idxs[item] = cluster_idx

# q6_str = array_to_str(idxs)
# write('q6.txt', q6_str)
# del q6_str

In [17]:
a = np.arange(50).reshape((-1, 5))

b = np.random.default_rng().choice(
    a,
    size=2,
    replace=False,
    shuffle=False
)

distances = np.empty((a.shape[0], b.shape[0]))
print('b[0]:', b[0], sep='\n', end='\n\n')
print('a - b[0]:', a - b[0], sep='\n', end='\n\n')
print('(a - b[0]) ** 2:', (a - b[0]) ** 2, sep='\n', end='\n\n')
print('np.sum((a - b[0]) ** 2, axis=1):', np.sum((a - b[0]) ** 2, axis=1), sep='\n', end='\n\n')

for i, _b in enumerate(b):
    print(_b, i)
    distances[:, i] = np.sum((a - _b) ** 2, axis=1)
print()
print('distances:', distances, sep='\n', end='\n\n')
print('np.argmin(distances, axis=1):', np.argmin(distances, axis=1), sep='\n', end='\n\n')

b[0]:
[30 31 32 33 34]

a - b[0]:
[[-30 -30 -30 -30 -30]
 [-25 -25 -25 -25 -25]
 [-20 -20 -20 -20 -20]
 [-15 -15 -15 -15 -15]
 [-10 -10 -10 -10 -10]
 [ -5  -5  -5  -5  -5]
 [  0   0   0   0   0]
 [  5   5   5   5   5]
 [ 10  10  10  10  10]
 [ 15  15  15  15  15]]

(a - b[0]) ** 2:
[[900 900 900 900 900]
 [625 625 625 625 625]
 [400 400 400 400 400]
 [225 225 225 225 225]
 [100 100 100 100 100]
 [ 25  25  25  25  25]
 [  0   0   0   0   0]
 [ 25  25  25  25  25]
 [100 100 100 100 100]
 [225 225 225 225 225]]

np.sum((a - b[0]) ** 2, axis=1):
[4500 3125 2000 1125  500  125    0  125  500 1125]

[30 31 32 33 34] 0
[25 26 27 28 29] 1

distances:
[[4500. 3125.]
 [3125. 2000.]
 [2000. 1125.]
 [1125.  500.]
 [ 500.  125.]
 [ 125.    0.]
 [   0.  125.]
 [ 125.  500.]
 [ 500. 1125.]
 [1125. 2000.]]

np.argmin(distances, axis=1):
[1 1 1 1 1 1 0 0 0 0]



In [18]:
class KMeans:
    class Clustering:
        @staticmethod
        def assign_points(centroids: np.ndarray, X: np.ndarray) -> np.ndarray:
            distance_matrix = KMeans.Clustering.compute_distances(centroids, X)
            return np.argmin(distance_matrix, axis=1)
        
        @staticmethod
        def choose_centroids(X: np.ndarray, k: int = 2) -> np.ndarray:
            return np.random.default_rng().choice(
                X,
                size=k,
                replace=False,
                shuffle=False
            )

        @staticmethod
        def compute_centroids(X: np.ndarray, assignments: np.ndarray, k: int) -> np.ndarray:
            classes = np.unique(assignments)
            
            centroids = np.empty((max(k, classes.shape[0]), X.shape[1]))
            
            for i, class_ in enumerate(classes):
                idxs = assignments == class_
                centroids[i] = np.mean(X[idxs], axis=0)
            
            return centroids
        
        @staticmethod
        def compute_distances(centroids: np.ndarray, X: np.ndarray) -> np.ndarray:
            distances = np.empty((X.shape[0], centroids.shape[0]))
            for i, c in enumerate(centroids):
                distances[:, i] = np.sum((X - c) ** 2, axis=1)
            
            return distances
    
    def __init__(self, k: int = 2, max_iters: int = 100, delta: float = 1e-5, ):
        self._k = k
        self._max_iters = max_iters
        self._delta = abs(delta)
    
    def _build(self, X: np.ndarray):
        ctr = 0
        intial_centroids = KMeans.Clustering.choose_centroids(X, self.k)
        
        while ctr < self._max_iters:  # avoid taking too much time
            ctr += 1
        
            assignment = KMeans.Clustering.assign_points(intial_centroids, X)
            updated_centroids = KMeans.Clustering.compute_centroids(X, assignment, self._k)
            
            if np.all(np.isclose(intial_centroids, updated_centroids)):
                break
            
            if np.all(np.abs(updated_centroids - intial_centroids) < self._delta):
                break
            
            initial_centroids = updated_centroids
        
        self._centroids = updated_centroids
    
    def fit(self, X: np.ndarray) -> Self:
        self._build(X)
        return self
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        return KMeans.Clustering.assign_points(self._centroids, X)
    
    @property
    def centroids(self):
        return self._centroids
    
    @centroids.setter
    def centroids(self):
        raise ValueError('The `centroids` attribute should not be modified externally!')
    
    @property
    def k(self):
        return self._k
    
    @k.setter
    def k(self):
        raise ValueError('The `k` attribute should not be modified externally!')
    
    def distortion(self, X: np.ndarray) -> np.ndarray:
        _distortion = 0
        
        assignments = self.predict(X)
        
        classes = np.unique(assignments)
        
        for class_ in classes:
            idxs = assignments == class_
            _distortion += np.sum((X[idxs] - self._centroids[class_]) ** 2)
        
        return _distortion

In [19]:
kmeans = KMeans(k=5)
kmeans.fit(parametric_data)

In [20]:
predictions = kmeans.predict(parametric_data)
predictions

q7_str = array_to_str(predictions)
write('q7.txt', q7_str)

In [21]:
centroids = kmeans.centroids

q8_str = array_to_str(centroids)
write('q8.txt', q8_str)

In [26]:
distortion = kmeans.distortion(parametric_data)

q9_str = str(np.around(distortion, 4))
write('q9.txt', q9_str)